**Franco Ortiz** :Data scientist

UFO-reports-analysis
# Problem Statement
### Questions to answer
- What time is more frequent to see them?
- How long did the experience take?
- What shapes are most common and where?
- Top words that described that moment (wordcloud)
- heat map with quantities and cities for number of sightings (geopandas and foliumn)

# Data importing
#### modules

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from dataprep.clean import clean_date
import re

# set seed for reproducibility
np.random.seed(0)

In [2]:
df = pd.read_csv("data/ufo.csv", keep_default_na=True)
df.head(20)

Date / Time                                     City  \
0                             00:00                Dieppe (Moncton) (Canada)   
1                          4/ 23:22                            Atlantic City   
2                    03142020 23:04                                  Norwich   
3                       1970s 23:00                            Grande Summit   
4                             23:00                    Desolation Wilderness   
5                             23:00                            Playa del Rey   
6                             23:00                        Milltown (Canada)   
7                             23:00                                  Manning   
8                             23:00                           New Alexandria   
9                             23:00  Northern California Coast (in, highway)   
10                            23:00                  14 Island Lake (Canada)   
11                   04/00/80 23:00                                 Cohoctah   
12                            22:55                       Corby (UK/England)   
13                            22:50                     Necochea (Argentina)   
14                   O6/03/19 22:44                               Louisville   
15                   09/16/01 22:38                           Bonita Springs   
16  hudson river = town of wa 22:34                          Washingtonville   
17                   01/15/70 22:30               Melbourne (VIC, Australia)   
18                   00/00/99 22:30                 Iron Bridge (UK/England)   
19                            22:30                             Mesa Arizona   

   State         Country      Shape          Duration  \
0     NB          Canada       Oval               NaN   
1     NJ             USA       Disk        30 minutes   
2     VT             USA     Circle        97 seconds   
3     PA             USA      Light         5 minutes   
4     CA             USA   Triangle            2 min.   
5     CA             USA   Fireball       3-5 seconds   
6     NB          Canada     Circle               NaN   
7     SC             USA     Sphere         2 minutes   
8     PA             USA      Other        15 minutes   
9     CA             USA      Cigar          3-5 min.   
10    QC          Canada   Fireball         ~2-3 sec.   
11    MI             USA    Unknown            15 min   
12   NaN  United Kingdom      Cigar     10.to 15.mins   
13   NaN       Argentina  Formation        10 seconds   
14    KY             USA      Light        30 seconds   
15    FL             USA     Circle            2hours   
16    NY             USA       Disk  1hour 15 minutes   
17    VI       Australia     Circle           45 mins   
18   NaN  United Kingdom      Light             2mins   
19    AZ             USA      Light        20 minutes   

                                              Summary    Posted Images  
0   ((HOAX??)) we wher looking .. in the sky in di...   9/24/12    NaN  
1   Observed a very bright white saucer (had two l...    5/6/17    NaN  
2                         Strange sighting in the sky   5/15/20    NaN  
3     4 lights speeding very fast across sky silently   8/16/21    NaN  
4   triangle of lights moving slowly through the n...   9/15/05    NaN  
5   Neon green, long-tailed fireball meteor-lookin...    7/8/04    NaN  
6   I wonder where they are today, and if they are...    6/4/04    NaN  
7   The object was metallic in color. Spherical in...   6/25/20    NaN  
8                                    Translucent UFO.    5/9/19    NaN  
9   We were travelling along the coast of Californ...   10/2/99    NaN  
10  A 2-3 second streak of an emerald green laser-...    6/4/04    NaN  
11  I Know this was 20 years ago. It still bothers...   11/9/99    NaN  
12  I WAS LOCKING UP MY PIGEONS FOR THE NIGHT.BRIG...   2/24/01    NaN  
13                   Crafts in the South of the World   8/28/02    NaN  
14  Thought it was a plane at first but it was mov

- ***As we see index 1,2,3,4 must be put in an standard format***

In [3]:
df.columns

Index(['Date / Time', 'City', 'State', 'Country', 'Shape', 'Duration',
       'Summary', 'Posted', 'Images'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138298 entries, 0 to 138297
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Date / Time  138197 non-null  object
 1   City         137909 non-null  object
 2   State        128886 non-null  object
 3   Country      138075 non-null  object
 4   Shape        132096 non-null  object
 5   Duration     131537 non-null  object
 6   Summary      138227 non-null  object
 7   Posted       138296 non-null  object
 8   Images       524 non-null     object
dtypes: object(9)
memory usage: 9.5+ MB


# Data Cleaning

In [5]:
df['Date / Time']

0                  00:00
1               4/ 23:22
2         03142020 23:04
3            1970s 23:00
4                  23:00
               ...      
138293      5/1/22 06:00
138294      5/1/22 04:41
138295      5/1/22 04:25
138296      5/1/22 04:00
138297      5/1/22 02:37
Name: Date / Time, Length: 138298, dtype: object

#### splitting datetime column

In [6]:
# Create a copy of the dataframe
df_clean = df.copy()

# Try to convert the 'Date / Time' column to datetime format
df_clean['Date / Time'] = pd.to_datetime(df_clean['Date / Time'], errors='coerce')

# Check for invalid dates and handle them
df_clean = df_clean[pd.notnull(df_clean['Date / Time'])]

# Extract the date and time separately into separate columns
df_clean['Date'] = df_clean['Date / Time'].dt.date
df_clean['Time'] = df_clean['Date / Time'].dt.time

# Drop the original 'Date / Time' column
df_clean.drop(columns=['Date / Time'], inplace=True)

#### column values to lowercase

In [7]:
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = df[column].str.lower()
df.head(10)

Date / Time                                     City State Country  \
0           00:00                dieppe (moncton) (canada)    nb  canada   
1        4/ 23:22                            atlantic city    nj     usa   
2  03142020 23:04                                  norwich    vt     usa   
3     1970s 23:00                            grande summit    pa     usa   
4           23:00                    desolation wilderness    ca     usa   
5           23:00                            playa del rey    ca     usa   
6           23:00                        milltown (canada)    nb  canada   
7           23:00                                  manning    sc     usa   
8           23:00                           new alexandria    pa     usa   
9           23:00  northern california coast (in, highway)    ca     usa   

      Shape     Duration                                            Summary  \
0      oval          NaN  ((hoax??)) we wher looking .. in the sky in di...   
1      disk   30 minutes  observed a very bright white saucer (had two l...   
2    circle   97 seconds                        strange sighting in the sky   
3     light    5 minutes    4 lights speeding very fast across sky silently   
4  triangle       2 min.  triangle of lights moving slowly through the n...   
5  fireball  3-5 seconds  neon green, long-tailed fireball meteor-lookin...   
6    circle          NaN  i wonder where they are today, and if they are...   
7    sphere    2 minutes  the object was metallic in color. spherical in...   
8     other   15 minutes                                   translucent ufo.   
9     cigar     3-5 min.  we were travelling along the coast of californ...   

    Posted Images  
0  9/24/12    NaN  
1   5/6/17    NaN  
2  5/15/20    NaN  
3  8/16/21    NaN  
4  9/15/05    NaN  
5   7/8/04    NaN  
6   6/4/04    NaN  
7  6/25/20    NaN  
8   5/9/19    NaN  
9  10/2/99    NaN

#### cleaning Summary column

In [8]:
# return only words if them are str, if not '', using regular expression
# (r'\b\w+\b', x) words and digits
df['words'] = df['Summary'].apply(lambda x: re.findall(r'[^\d\W]+', x) if isinstance(x, str) else [])
df['words']

0         [hoax, we, wher, looking, in, the, sky, in, di...
1         [observed, a, very, bright, white, saucer, had...
2                         [strange, sighting, in, the, sky]
3         [lights, speeding, very, fast, across, sky, si...
4         [triangle, of, lights, moving, slowly, through...
                                ...                        
138293    [two, lights, shaped, like, dots, flying, abov...
138294    [red, dots, with, a, white, blur, glow, surrou...
138295    [two, golden, orbs, movin, slowly, one, a, bit...
138296    [object, observed, flying, southwest, to, nort...
138297    [objects, off, flight, path, rotating, most, l...
Name: words, Length: 138298, dtype: object

#### cleaning Duration column

In [9]:
df['Duration'] = df['Duration'].astype(str)
df['Duration'] = df['Duration'].str.extract(r'(\d+)').astype(float)
df['Duration'] = df['Duration'].fillna(0)
df['Duration'] = df['Duration'].apply(lambda x: x * 60 if x < 100 else x)
df['Duration'] = df['Duration'].astype(int)
df['Duration'].head(10)

0       0
1    1800
2    5820
3     300
4     120
5     180
6       0
7     120
8     900
9     180
Name: Duration, dtype: int32

#### removing unuseful chars in city state country

In [17]:
df['Location'] = df['City'] + ', ' + df['State'] + ', ' + df['Country']

In [19]:
df['Location_Info'] = df['Location'].apply(lambda x: [word for word in re.findall(r'(\w+)', x) if not word.isdigit()])
df['Location_Info'].head(11)

0                 [dieppe, moncton, canada, nb, canada]
1                             [atlantic, city, nj, usa]
2                                    [norwich, vt, usa]
3                             [grande, summit, pa, usa]
4                     [desolation, wilderness, ca, usa]
5                            [playa, del, rey, ca, usa]
6                        [milltown, canada, nb, canada]
7                                    [manning, sc, usa]
8                            [new, alexandria, pa, usa]
9     [northern, california, coast, in, highway, ca,...
10                   [island, lake, canada, qc, canada]
Name: Location_Info, dtype: object

# EDA

### what shapes are most common and where 

In [ ]:
sns.countplot(y ='Shape', 
              data = df,
              color='limegreen',
              order = df['Shape'].value_counts().index)
plt.show()

In [ ]:
# Top 15 places
shapes_where = df.groupby(['City'])['Shape'].value_counts().sort_values(ascending=False).head(15)
shapes_where

- ***Seems like most common shape is 'light-shaped', followe by circle and triangle. The top 3 places they are seen is in Seattle, Phoenix, Las Vegas***

### Most used words in description
Text Mining with NLTK, and frecuency distribution

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import PIL.Image

In [ ]:
# for WordCloud.generate() method should be a single string
words_string = ' '.join(df['words'].str.join(' '))

In [ ]:
wc = WordCloud(background_color='white', max_font_size=40)
wc.generate(words_string)

In [ ]:
personalized_stopwords = ["nuforc", "one", "two", "three", "see", "saw", "seen", "looked", "light", "white light", "red light"]

In [ ]:
mask = np.array(PIL.Image.open("img/ufo.jpg"))

wc = WordCloud(stopwords = STOPWORDS.union(set(personalized_stopwords)),
               mask = mask,
               background_color = "white",
               contour_color = "black",
               contour_width = 0.9,
               min_font_size=5).generate(words_string)

plt.imshow(wc.to_array(), interpolation='bilinear')
plt.axis("off")
plt.show()

- ***As we can see most frequent words are :*** 
        ***Sky, Object, bright light, star, hovering, sound, ufo, night sky***

#### What time is most frequent to see them?

In [ ]:
top_15_times = df_clean['Time'].value_counts().sort_values(ascending=False).head(15)

In [ ]:
top_15_times.plot(kind='bar',figsize=(7, 3),color='limegreen')

plt.xlabel('Time')
plt.ylabel('Counts')
plt.title('Top 15 Time with highest counts')
plt.xticks(rotation=50)

plt.show()

- ***most frequent hours are from 21 to 23 hs***

#### How long did the experience take?

In [ ]:
df.head()

In [ ]:
#Count the occurrences of each value in the Duration column
sorted_duration = df.sort_values(by='Duration')
value_counts = sorted_duration['Duration'].value_counts()

#Create the violin plot
value_counts.plot(kind='bar',figsize=(7, 3),
                 color='limegreen')
plt.xlim(0, 15)

#Add a title and axis labels
plt.title("Distribution of UFO Sighting Durations")
plt.xlabel("Duration (in minutes)")
plt.ylabel("Count")
plt.xticks(rotation=50)

#Show the plot
plt.show()

In [ ]:
print('mean duration: ',value_counts.mean())
print('minimun duration: ',value_counts.min())

- ***duration tends to be:  300 seconds ->  around 5 min***
- ***mean duration:         794 seconds ->13 min***
- ***minimun duration :     1 second***

# Modelling

#### Heat map with quantities and cities for number of sightings

In [ ]:
df_clean.head()

- To plot cities in GeoPandas, you need to have latitude and longitude information for each city. If you don't have this information in your dataframe, you can obtain it using a geocoding library

In [ ]:
import geopandas as gpd
import folium
from geopy.geocoders import Nominatim
import time

# instantiate 
geolocator = Nominatim(user_agent="geoapiExercises")

# checker and returning execpt API not responding and timing out,
def get_lat_long(city, state, country):
    for i in range(3):
        try:
            location = geolocator.geocode(df['Location_Info'])
            if location:
                return (location.latitude, location.longitude)
            else:
                return (None, None)
        except:
            time.sleep(1)
    return (None, None)

df['lat_long'] = df.apply(lambda x: get_lat_long(x['City'], x['State'], x['Country']), axis=1)
df[['latitude', 'longitude']] = pd.DataFrame(df['lat_long'].tolist(), index=df.index)
df.drop(columns=['lat_long'], inplace=True)

- Once you have the latitude and longitude information, you can create a GeoDataFrame and plot it using matplotlib:

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

gdf.plot(marker='o', color='red', markersize=5)


# Communication